# Parsing WARC File

### 🛠 Installing Necessary Libraries

Toplam işlenen kayıt: 98984
Toplam geçen süre: 7211.72 saniye
Saniyede ortalama kayıt: 13.73

Bu kayıtlar için başlık, url ve html bilgileri çıkarılarak sqlite db'ye kaydedildi

In [9]:
pip install warcio beautifulsoup4 psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [10]:
# from warcio.archiveiterator import ArchiveIterator
# from bs4 import BeautifulSoup
# import sqlite3
# import time

# def process_warc_file_in_chunks(warc_path, db_path, batch_size=100):
#     """
#     WARC dosyasını parça parça işleyerek SQLite'a kaydeder
    
#     Args:
#         warc_path: WARC dosyasının yolu
#         db_path: SQLite veritabanının yolu
#         batch_size: Her defasında işlenecek kayıt sayısı
#     """
#     # SQLite bağlantısını oluştur
#     conn = sqlite3.connect(db_path)
#     c = conn.cursor()
    
#     # Tablo oluştur (eğer yoksa)
#     c.execute('CREATE TABLE IF NOT EXISTS pages (id INTEGER PRIMARY KEY, url TEXT, title TEXT, html TEXT)')
    
#     # İşleme istatistikleri
#     start_time = time.time()
#     total_records = 0
#     batch_records = []
    
#     # WARC dosyasını aç ve kayıtları işle
#     with open(warc_path, 'rb') as stream:
#         for record in ArchiveIterator(stream):
#             if record.rec_type == 'response':
#                 try:
#                     url = record.rec_headers.get_header('WARC-Target-URI')
#                     raw_html = record.content_stream().read().decode('utf-8', errors='replace')
                    
#                     soup = BeautifulSoup(raw_html, 'html.parser')
#                     title = soup.title.string if soup.title else ''
                    
#                     batch_records.append((url, title, raw_html))
#                     total_records += 1
                    
#                     # Belirli bir sayıda kayıt toplandıysa veritabanına kaydet
#                     if len(batch_records) >= batch_size:
#                         c.executemany('INSERT INTO pages (url, title, html) VALUES (?, ?, ?)', batch_records)
#                         conn.commit()
                        
#                         elapsed_time = time.time() - start_time
#                         print(f"İşlenen kayıt: {total_records}, Geçen süre: {elapsed_time:.2f} saniye")
                        
#                         # Belleği temizle
#                         batch_records = []
                
#                 except Exception as e:
#                     print(f"Hata oluştu: {e} - URL: {url if 'url' in locals() else 'bilinmiyor'}")
    
#     # Kalan kayıtları kaydet
#     if batch_records:
#         c.executemany('INSERT INTO pages (url, title, html) VALUES (?, ?, ?)', batch_records)
#         conn.commit()
    
#     # İstatistikleri göster
#     total_time = time.time() - start_time
#     print(f"Toplam işlenen kayıt: {total_records}")
#     print(f"Toplam geçen süre: {total_time:.2f} saniye")
#     print(f"Saniyede ortalama kayıt: {total_records/total_time:.2f}")
    
#     conn.close()

# # Kodu çalıştır
# warc_file = "./website_data.warc"
# db_file = "websites.db"
# process_warc_file_in_chunks(warc_file, db_file, batch_size=500)

### WARC dosyasının parse edilip PostgreSQL DB'ye kaydedilmesi

In [11]:
from warcio.archiveiterator import ArchiveIterator
from bs4 import BeautifulSoup
import psycopg2
import time
from psycopg2.extras import execute_values
from tqdm import tqdm  # tqdm kütüphanesini ekleyin

def process_warc_file(warc_path, db_params, batch_size=100):
    """
    WARC dosyasını parça parça işleyerek PostgreSQL'e kaydeder
    
    Args:
        warc_path: WARC dosyasının yolu
        db_params: PostgreSQL bağlantı parametreleri (dict)
        batch_size: Her defasında işlenecek kayıt sayısı
    """
    # PostgreSQL bağlantısını oluştur
    conn = psycopg2.connect(**db_params)
    print("connection success", conn)
    cur = conn.cursor()
    print("cursor success", cur)
    
    # İşleme istatistikleri
    start_time = time.time()
    total_records = 0
    batch_records = []
    
    # WARC dosyasını aç ve kayıtları işle
    with open(warc_path, 'rb') as stream:
        # tqdm ile ilerleme çubuğunu ekleyin
        print("started processing warc file");
        for record in tqdm(ArchiveIterator(stream), desc="İşlenen Kayıtlar"):
            if record.rec_type == 'response':
                try:
                    url = record.rec_headers.get_header('WARC-Target-URI')
                    raw_html = record.content_stream().read().decode('utf-8', errors='replace')
                    # remove null characters
                    raw_html = raw_html.replace('\x00', '')
                    
                    soup = BeautifulSoup(raw_html, 'html.parser')
                    title = soup.title.string if soup.title else ''
                    
                    batch_records.append((url, title, raw_html))
                    total_records += 1
                    
                    # Belirli bir sayıda kayıt toplandıysa veritabanına kaydet
                    if len(batch_records) >= batch_size:
                        print("inserting records to db", len(batch_records));
                        execute_values(cur,
                            'INSERT INTO pages (url, title, html) VALUES %s',
                            batch_records,
                            template='(%s, %s, %s)'
                        )
                        conn.commit()
                        
                        # Belleği temizle
                        batch_records = []
            
                except Exception as e:
                    print(f"Hata oluştu: {e} - URL: {url if 'url' in locals() else 'bilinmiyor'}")
    
    # Kalan kayıtları kaydet
    if batch_records:
        execute_values(cur,
            'INSERT INTO pages (url, title, html) VALUES %s',
            batch_records,
            template='(%s, %s, %s)'
        )
        conn.commit()
    
    # İstatistikleri göster
    total_time = time.time() - start_time
    print(f"Toplam işlenen kayıt: {total_records}")
    print(f"Toplam geçen süre: {total_time:.2f} saniye")
    print(f"Saniyede ortalama kayıt: {total_records/total_time:.2f}")
    
    cur.close()
    conn.close()

In [12]:
# PostgreSQL bağlantı parametreleri
db_params = {
    'dbname': 'websites',
    'user': 'postgres',
    'password': '1234567',
    'host': 'localhost',
    'port': '5432'
}

# WARC dosyasının yolu
warc_file = "../website_data.warc"

# Kodu çalıştır
process_warc_file(warc_file, db_params, batch_size=500)

connection success <connection object at 0x10d2a2650; dsn: 'user=postgres password=xxx dbname=websites host=localhost port=5432', closed: 0>
cursor success <cursor object at 0x115a9d030; closed: 0>
started processing warc file


İşlenen Kayıtlar: 0it [00:00, ?it/s]

İşlenen Kayıtlar: 497it [00:41,  5.05it/s]

inserting records to db 500


İşlenen Kayıtlar: 999it [01:22, 18.14it/s]

...

Hata oluştu: A string literal cannot contain NUL (0x00) characters. - URL: https://www.zdf.de/nachrichten/heute-sendungen/syrien-baerbock-besuch-keine-finanzversprechen-erwartet-video-100.html
inserting records to db 614


İşlenen Kayıtlar: 1615it [04:25,  1.30s/it]

Hata oluştu: A string literal cannot contain NUL (0x00) characters. - URL: https://www.insider.gr/toyrismos/348840/i-skopelos-anamesa-stoys-10-proorismoys-tis-eyropis-me-tis-kalyteres-paralies-gia
inserting records to db 615


İşlenen Kayıtlar: 1615it [04:25,  6.07it/s]


KeyboardInterrupt: 